In [4]:
from datasets import load_dataset

spam_detection_dataset = load_dataset("Deysi/spam-detection-dataset")

Generating train split:   0%|          | 0/8175 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2725 [00:00<?, ? examples/s]

In [5]:
spam_detection_dataset.set_format(type='pandas')

In [6]:
train_df = spam_detection_dataset['train'][:]
test_df = spam_detection_dataset['test'][:]

In [7]:
train_df.label.value_counts(normalize=True)

label
spam        0.504587
not_spam    0.495413
Name: proportion, dtype: float64

In [8]:
test_df.label.value_counts(normalize=True)

label
spam        0.504587
not_spam    0.495413
Name: proportion, dtype: float64

In [9]:
train_df = train_df.sample(1500, random_state=10)
test_df = test_df.sample(500, random_state=10)

In [11]:
conda install joblib

Channels:
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - joblib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    joblib-1.4.2               |  py312h06a4308_0         513 KB
    ------------------------------------------------------------
                                           Total:         513 KB

The following NEW packages will be INSTALLED:

  joblib             pkgs/main/linux-64::joblib-1.4.2-py312h06a4308_0 



                                                                                
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [13]:
from joblib import dump

train_df.to_csv('dataset/train_df.csv', index=False)
test_df.to_csv('dataset/test_df.csv', index=False)

In [16]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 471.1 kB/s eta 0:00:001m689.3 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.2 MB/s eta 0:00:003.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.3 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.4 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 38.6 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.3 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 23.0 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━

In [18]:
from sentence_transformers import SentenceTransformer

# Load the model
sentence_model = SentenceTransformer('all-mpnet-base-v2', device='cpu')

In [19]:
train_embeddings = sentence_model.encode(train_df['text'].values, show_progress_bar=True, batch_size=32)

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

In [20]:
test_embeddings = sentence_model.encode(test_df['text'].values, show_progress_bar=True, batch_size=32)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [22]:
dump(train_embeddings, 'dataset/embeddings/train_embeddings.joblib')
dump(test_embeddings, 'dataset/embeddings/test_embeddings.joblib')

['dataset/embeddings/test_embeddings.joblib']

In [23]:
# import randomforestclassifier from sklearn
from sklearn.ensemble import RandomForestClassifier

In [24]:
clf = RandomForestClassifier()

In [25]:
clf.fit(train_embeddings, train_df['label'])

RandomForestClassifier()

In [26]:
predict = clf.predict(test_embeddings)

In [27]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_df['label'], predict)

array([[247,   1],
       [  0, 252]])

In [28]:
from sklearn.metrics import classification_report

print(classification_report(test_df['label'], predict))

              precision    recall  f1-score   support

    not_spam       1.00      1.00      1.00       248
        spam       1.00      1.00      1.00       252

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



In [30]:
pip install mlflow

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 1.5 MB/s eta 0:00:002.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 32.0 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 38.1 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 2.0 MB/s e

In [ ]:
!mlflow ui --backend-store-uri sqlite:///mlflow.db

In [ ]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
MLFLOW_EXPERIMENT_NAME = "email-spam-detection-experiment"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# Create a new experiment
client.create_experiment(name=MLFLOW_EXPERIMENT_NAME)

In [36]:
pip install optuna_integration

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 978.6 kB/s eta 0:00:001m1.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [37]:
import optuna
import mlflow
from optuna_integration.mlflow import MLflowCallback

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

<Experiment: artifact_location='/workspaces/Email-spam-detection/mlruns/1', creation_time=1720705963584, experiment_id='1', last_update_time=1720705963584, lifecycle_stage='active', name='email-spam-detection-experiment', tags={}>

In [38]:
from sklearn.metrics import accuracy_score

def objective(trial):
    rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
    rf_n_estimators = trial.suggest_int("rf_n_estimators", 5, 100, log=True)
    clf = RandomForestClassifier(max_depth=rf_max_depth, n_estimators=rf_n_estimators)
    clf.fit(train_embeddings, train_df['label'])

    predictions = clf.predict(test_embeddings)
    accuracy = accuracy_score(test_df['label'], predictions)
    return accuracy

mlflc = MLflowCallback(
    tracking_uri=MLFLOW_TRACKING_URI,
    metric_name='accuracy',
    create_experiment=False,
)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, callbacks=[mlflc])

/tmp/ipykernel_7585/2127476819.py:13: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2024-07-11 13:57:19,513] A new study created in memory with name: no-name-2fbfbd86-bb79-4f0b-b2ac-0c5414a82b77
[I 2024-07-11 13:57:19,713] Trial 0 finished with value: 0.994 and parameters: {'rf_max_depth': 4, 'rf_n_estimators': 14}. Best is trial 0 with value: 0.994.
[I 2024-07-11 13:57:21,050] Trial 1 finished with value: 0.998 and parameters: {'rf_max_depth': 8, 'rf_n_estimators': 53}. Best is trial 1 with value: 0.998.
[I 2024-07-11 13:57:21,400] Trial 2 finished with value: 0.984 and parameters: {'rf_max_depth': 21, 'rf_n_estimators': 9}. Best is trial 1 with value: 0.998.
[I 2024-07-11 13:57:22,498] Trial 3 finished with value: 0.996 and parameters: {'rf_max_depth': 32, 'rf_n_estimators': 43}. Best is trial 1 with value: 0.998.
[I 2024-07-11 13:57:22,942] Trial 4 finished with value: 0.998 and param

In [39]:
study.best_value

1.0

In [40]:
# Search for runs in the experiment
# Get based on the best trial value with the lowest n_estimators
from mlflow.entities import ViewType

spam_detection_experiment=dict(mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME))
experiment_id=spam_detection_experiment['experiment_id']

best_run = client.search_runs( 
    experiment_ids=experiment_id,
    filter_string=f'metrics.accuracy = {study.best_value}',
    run_view_type= ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=['parameters.rf_n_estimators ASC']
)[0]

In [42]:
best_max_depth = int(best_run.data.params['rf_max_depth'])
best_n_estimators = int(best_run.data.params['rf_n_estimators'])

mlflow.log_params({'rf_max_depth': best_max_depth, 'rf_n_estimators': best_n_estimators})

best_clf = RandomForestClassifier(n_estimators=best_n_estimators, max_depth=best_max_depth)
best_clf.fit(train_embeddings, train_df['label'])

best_predictions = best_clf.predict(test_embeddings)
accuracy = accuracy_score(test_df['label'], best_predictions)
mlflow.log_metric("accuracy", accuracy)

dump(best_clf, 'models/best_clf.joblib')
mlflow.sklearn.log_model(best_clf, artifact_path="models", registered_model_name='spam-detector')

Successfully registered model 'spam-detector'.
Created version '1' of model 'spam-detector'.
